# 6

In [12]:
import math
import operator
import copy

In [15]:
class Node:
    def __init__(self, ID, score):
        self.ID = ID
        self.score = score
        self.adjacents = set()
        
    def out_degree(self):
        return len(self.adjacents)
    
    def __eq__(self, other):
        """Override the default Equals behavior"""
        if isinstance(other, self.__class__):
            return self.ID == other.ID
        return False
    
    def __ne__(self, other):
        """Define a non-equality test"""
        return not self.__eq__(other)
    
    def __str__(self):
        return '(%s, %s)' % (self.ID, self.score)
    
    def __repr__(self):
        return self.__str__()
    
    def __hash__(self):
        return self.ID

In [38]:
class Graph:
    def __init__(self, N, fully_undirected=False):
        self.nodes = []
        self.N = N
        self.fully_undirected = fully_undirected
        for i in range(N):
            # initializes page rank algorithm.
            self.nodes.append(Node(i, 1.0 / float(N)))
            
    def add_edge(self,i,j):
        self.nodes[i].adjacents.add(self.nodes[j])
        
    def add_edge_und(self,i,j):
        self.add_edge(i, j)
        self.add_edge(j, i)
    
    #Little trick for the facebook graph. performance related
    def get_nodes_in(self, i):
        if self.fully_undirected:
            return self.nodes[i].adjacents
        else:
            return [node for node in self.nodes if self.nodes[i] in node.adjacents]
        
    def scores(self):
        d = {}
        for node in self.nodes:
            d[node.ID] = node.score
        return d
    
    def update_scores(self, scores_dict):
        for ID, score in scores_dict.items():
            self.nodes[ID].score = score
    
    def __str__(self):
        s = ""
        for i in range(len(self.nodes)):
            s+='%s: %s \n' %(i, self.nodes[i].adjacents)
        return s
    
    def __repr__(self):
        return self.__str__()

In [10]:
def read_graph(filepath, undirected = False):
    with open(filepath) as fileIn:
        N = int(fileIn.readline())
        g = Graph(N, fully_undirected=undirected)
        for line in fileIn:
            i, j = (int(s) for s in line.split())
            if undirected:
                g.add_edge_und(i,j)
            else:
                g.add_edge(i,j)
        return g

In [115]:
class PageRank:
    def __init__(self, graph, epsilon = 0.00000001, rounds = math.inf, e = 1.0/7):
        self.graph = graph
        self.epsilon = epsilon
        self.rounds = rounds
        self.e = e
    
    # it can terminate based on epsilon change or number of iterations
    def converge(self, update_graph=False, scaled_page_rank = True):
        _break = False
        _round = 0
        _scores = self.graph.scores()
        while _round < self.rounds and not _break:
            _break = True
            _round_scores = copy.copy(_scores)
            for node in self.graph.nodes:
                new_score = self.__page_rank_score(node.ID, _scores)
                if scaled_page_rank:
                    new_score = (self.e / self.graph.N) + ((1 - self.e) * new_score)
                if (abs(new_score - _scores[node.ID])) > self.epsilon:
                    _break = False
                _round_scores[node.ID] = new_score
            _round += 1
            _scores = _round_scores
        
        if update_graph:
            self.graph.update_scores(_scores)
        return _scores, _round 
    
    def __page_rank_score(self, i, scores):
        in_nodes = self.graph.get_nodes_in(i)
        scores = [ scores[node.ID] / node.out_degree() for node in in_nodes]
        return sum(scores)

 #### We can use `n` to stop the iterations or it will stop when the change in scores is very small.

##### Fig 11_1_1 - Triangle attached to self loop node z. fig 11.1 (left)

In [106]:
# read the graph
g = read_graph('f_11_1_1.txt')
# prepare for page rank
page_rank = PageRank(graph = g, e = 1.0/2)
# try standard page rank. We will expect a tendency towards 1 for node 3(z) and 0 for the rest
scores, niter = page_rank.converge(scaled_page_rank = False)
print("Standard page rank. Iterations: %s. Scores-> %s" % (niter, scores))

print("\n")
# Now we try scaled page rank.
scores, niter = page_rank.converge(scaled_page_rank = True)
print("Scaled page rank. Iterations: %s. Scores-> %s" % (niter, scores))

Standard page rank. Iterations: 73. Scores-> {0: 1.4901161193847656e-08, 1: 7.450580596923828e-09, 2: 1.4901161193847656e-08, 3: 0.999999962747097}


Scaled page rank. Iterations: 19. Scores-> {0: 0.23333333432674408, 1: 0.18333333358168602, 2: 0.21666666865348816, 3: 0.36666666343808174}


##### Fig 11_1_2 - Triangle attached to two nodes that loop around each other (fig 11.1 right)

In [107]:
# read the graph
g = read_graph('f_11_1_2.txt')
# prepare for page rank
page_rank = PageRank(graph = g, e = 1.0/2)
# try standard page rank. We will expect a tendency towards 0.5 for nodes 3 and 4 and 0 for the rest.
scores, niter = page_rank.converge(scaled_page_rank = False)
print("Standard page rank. Iterations: %s. Scores-> %s" % (niter, scores))

print("\n")
# Now we try scaled page rank.
scores, niter = page_rank.converge(scaled_page_rank = True)
print("Scaled page rank. Iterations: %s. Scores-> %s" % (niter, scores))

Standard page rank. Iterations: 46. Scores-> {0: 1.3938343875251262e-08, 1: 4.646114625083754e-09, 2: 1.3938343875251262e-08, 3: 0.49999998373859866, 4: 0.49999998373859866}


Scaled page rank. Iterations: 16. Scores-> {0: 0.18260869783629116, 1: 0.1304347829727152, 2: 0.1652173956725823, 3: 0.2608695617592057, 4: 0.2608695617592057}


##### Fig 11_2 - two disjoint rectangles

In [111]:
# read the graph
g = read_graph('f_11_2.txt')
# prepare for page rank
page_rank = PageRank(graph = g, e = 1.0/7)
# try standard page rank. We will expect a tendency towards 1/6 for all nodes.
scores, niter = page_rank.converge(scaled_page_rank = False)
print("Standard page rank. Iterations: %s. Scores-> %s" % (niter, scores))

# print("\n")
# Now we try scaled page rank. Result should be the same
scores, niter = page_rank.converge(scaled_page_rank = True)
print("Scaled page rank. Iterations: %s. Scores-> %s" % (niter, scores))

Standard page rank. Iterations: 1. Scores-> {0: 0.16666666666666666, 1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666, 4: 0.16666666666666666, 5: 0.16666666666666666}
Scaled page rank. Iterations: 1. Scores-> {0: 0.16666666666666666, 1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666, 4: 0.16666666666666666, 5: 0.16666666666666666}


##### Fig 11_3 - single triangle

In [112]:
# read the graph
g = read_graph('f_11_3.txt')
# prepare for page rank
page_rank = PageRank(graph = g, e = 1.0/7)
# try standard page rank. We will expect a tendency towards 1/6 for all nodes.
scores, niter = page_rank.converge(scaled_page_rank = False)
print("Standard page rank. Iterations: %s. Scores-> %s" % (niter, scores))

# print("\n")
# Now we try scaled page rank. Result should be the same
scores, niter = page_rank.converge(scaled_page_rank = True)
print("Scaled page rank. Iterations: %s. Scores-> %s" % (niter, scores))

Standard page rank. Iterations: 1. Scores-> {0: 0.3333333333333333, 1: 0.3333333333333333, 2: 0.3333333333333333}
Scaled page rank. Iterations: 1. Scores-> {0: 0.3333333333333333, 1: 0.3333333333333333, 2: 0.3333333333333333}


# 7

## a

In [113]:
#this is already taken care of by the way the graph is constructed.
fb_g = read_graph('facebook_combined.txt', undirected=True)
assert(len(fb_g.nodes[0].adjacents) == 347)

## b

#### We are looking at the average score as we iterate through the number of rounds. That should give us an idea of the tendency. We can see that after 4 rounds we converge.

In [117]:
# read the graph
fb_g = read_graph('facebook_combined.txt', undirected=True) 
for rounds in range(2, 20, 1):
        page_rank = PageRank(graph = fb_g, e = 1.0/7, rounds = rounds)
        scores, niter = page_rank.converge()
        print('Ran for %s iterations. Avg score for %s rounds : %s ' % (niter, rounds, sum(scores.values()) / len(scores)))

Ran for 2 iterations. Avg score for 2 rounds : 0.00024758603614756215 
Ran for 3 iterations. Avg score for 3 rounds : 0.0002475860361475614 
Ran for 4 iterations. Avg score for 4 rounds : 0.0002475860361475623 
Ran for 5 iterations. Avg score for 5 rounds : 0.00024758603614756075 
Ran for 6 iterations. Avg score for 6 rounds : 0.0002475860361475613 
Ran for 7 iterations. Avg score for 7 rounds : 0.0002475860361475612 
Ran for 8 iterations. Avg score for 8 rounds : 0.0002475860361475606 
Ran for 9 iterations. Avg score for 9 rounds : 0.00024758603614756183 
Ran for 10 iterations. Avg score for 10 rounds : 0.00024758603614756134 
Ran for 11 iterations. Avg score for 11 rounds : 0.0002475860361475615 
Ran for 12 iterations. Avg score for 12 rounds : 0.00024758603614756205 
Ran for 13 iterations. Avg score for 13 rounds : 0.0002475860361475616 
Ran for 14 iterations. Avg score for 14 rounds : 0.00024758603614756205 
Ran for 15 iterations. Avg score for 15 rounds : 0.0002475860361475613 
Ra

In [24]:
for i in range(2, 12, 2):
    
    fb_g.page_rank(rounds = i)
    scores = fb_g.scores()
    

Avg score for 2 rounds : 0.0002567400396618219 
Avg score for 4 rounds : 0.00025254916998110517 
Avg score for 6 rounds : 0.00025254916998110517 
Avg score for 8 rounds : 0.00025254916998110517 
Avg score for 10 rounds : 0.00025254916998110517 


#### And without rounds stop condition

In [118]:
# read the graph
fb_g = read_graph('facebook_combined.txt', undirected=True) 
page_rank = PageRank(graph = fb_g, e = 1.0/7)
scores, niter = page_rank.converge()
print('Ran for %s iterations. Avg score: %s ' % (niter,sum(scores.values()) / len(scores)))

Ran for 48 iterations. Avg score: 0.0002475860361475611 


## C

##### We can see that there is a big gap between the maximum score and the minimum score. That can mean that there are nodes that are the neuralgic centers of the networks, or influencers. We could use these nodes if we were to try and have full cascade in a contagion scenario.

In [123]:
# There doesn't seem to be a particular bucketing in scores.
print(max(scores.items(), key=operator.itemgetter(1)))
print(min(scores.items(), key=operator.itemgetter(1)))


(3437, 0.007569615371870586)
(2079, 3.9677335824609284e-05)


## d
#### This is definitely a measure of popularity. High scoring nodes are highly connected, which implies that they are very relevant or popular. However, there is a difference between been highly connected and highly influencial: one person can have many friends but  have very little influence on these friends.